In [ ]:
# Jinko specifics imports & initialization
# Please fold this section and do not change
import jinko_helpers as jinko

# Connect to Jinko (see README.md for more options)
jinko.initialize()

In [ ]:
# Cohere specific imports
import cohere
import os

# Connect to cohere
cohere_api_key = os.environ.get("COHERE_API_KEY")
co = cohere.ClientV2(cohere_api_key)

In [ ]:
# Cookbook specifics imports
import json

# Cookbook specifics constants:
# put here the constants that are specific to your cookbook like
# the reference to the Jinko items, the name of the model, etc.

## "Tool" aka function call use case

In this use case, we try to retrieve the coreItemId given the short id of the PI

### Step 1

The developer provides the get_core_item_id fuction to the model using the tools parameter.

Observe that, for each tool, the developer describes the tool name, description, and inputs. Each input can have a type and can be marked as required.



In [ ]:
# Function definitions
def wrap_get_core_item_id(shortId: str) -> dict:
    return jinko.get_core_item_id(shortId)["id"]


functions_map = {
    "wrap_get_core_item_id": wrap_get_core_item_id,
}

In [ ]:
# Tool definitions
tools = [
    {
        "type": "function",
        "function": {
            "name": "wrap_get_core_item_id",
            "description": "Retrieves the core item id of a project item from its short id.",
            "parameters": {
                "type": "object",
                "properties": {
                    "shortId": {
                        "type": "string",
                        "description": "The short id of the project item.",
                    }
                },
                "required": ["shortId"],
            },
        },
    },
]

In [ ]:
system_message = """
## Task & Context
You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or similar tools to help you, which you use to research your answer. You should focus on serving the user's needs as best you can, which will be wide-ranging.

## Style Guide
Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling.
"""

# user request

pi_short_id = (
    "cm-sIDm-CniF"  # this is a real short id of a cm already uploaded to jinko, in the api cookbooks project
)
user_message = "Please give me the core item id of PI with sid : " + str(pi_short_id)

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message},
]

### Step 2

The model’s response contains the tool plan, a list of appropriate tools to call in order to answer the user’s question, as well as the appropriate inputs for each tool call.



In [ ]:
response = co.chat(
    model="command-r-plus-08-2024",
    messages=messages,
    tools=tools,
)

print("The model recommends doing the following tool calls:\n")
print("Tool plan:")
print(response.message.tool_plan, "\n")
print("Tool calls:")
for tc in response.message.tool_calls:
    print(f"Tool name: {tc.function.name} | Parameters: {tc.function.arguments}")

# append the chat history
messages.append(
    {
        "role": "assistant",
        "tool_calls": response.message.tool_calls,
        "tool_plan": response.message.tool_plan,
    }
)

# Step 3

Now, the developer will query the appropriate tools and receive a tool result in return.



In [ ]:
# Iterate over the tool calls generated by the model
for tc in response.message.tool_calls:
    # here is where you would call the tool recommended by the model, using the parameters recommended by the model
    tool_result = functions_map[tc.function.name](**json.loads(tc.function.arguments))
    # store the output in a list
    tool_content = [{"type": "document", "document": {"data": tool_result}}]
    # Optional: add an "id" field in the "document" object, otherwise IDs are auto-generated
    # append the chat history
    messages.append({"role": "tool", "tool_call_id": tc.id, "content": tool_content})

    print("Tool results that will be fed back to the model in step 4:")
    for result in tool_content:
        print(result)

# Step 4

Call the chat endpoint again with the tool results for the model to generate the response with citations.




In [ ]:
response = co.chat(model="command-r-plus-08-2024", messages=messages, tools=tools)
print("Final answer:")
print(response.message.content[0].text)

# One function for everything

In [ ]:
# Function definitions
def wrap_get_core_item_id(shortId: str) -> dict:
    return jinko.get_core_item_id(shortId)["id"]


functions_map = {
    "wrap_get_core_item_id": wrap_get_core_item_id,
}

# Tool definitions
tools = [
    {
        "type": "function",
        "function": {
            "name": "wrap_get_core_item_id",
            "description": "Retrieves the core item id of a project item from its short id.",
            "parameters": {
                "type": "object",
                "properties": {
                    "shortId": {
                        "type": "string",
                        "description": "The short id of the project item.",
                    }
                },
                "required": ["shortId"],
            },
        },
    },
]


def run_chatbot(user_message):

    # Messages building
    system_message = """
    ## Task & Context
    You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or similar tools to help you, which you use to research your answer. You should focus on serving the user's needs as best you can, which will be wide-ranging.

    ## Style Guide
    Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling.
    """
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
    ]

    response = co.chat(
        model="command-r-plus-08-2024",
        messages=messages,
        tools=tools,
    )

    print(response.message.tool_plan, "\n")

    # append the chat history
    messages.append(
        {
            "role": "assistant",
            "tool_calls": response.message.tool_calls,
            "tool_plan": response.message.tool_plan,
        }
    )

    # Iterate over the tool calls generated by the model
    for tc in response.message.tool_calls:
        # here is where you would call the tool recommended by the model, using the parameters recommended by the model
        tool_result = functions_map[tc.function.name](**json.loads(tc.function.arguments))
        # store the output in a list
        tool_content = [{"type": "document", "document": {"data": tool_result}}]
        # Optional: add an "id" field in the "document" object, otherwise IDs are auto-generated
        # append the chat history
        messages.append({"role": "tool", "tool_call_id": tc.id, "content": tool_content})

        response = co.chat(model="command-r-plus-08-2024", messages=messages, tools=tools)
        print(response.message.content[0].text)



In [ ]:
user_message = "Please give me the core item id of PI with sid : cm-sIDm-CniF"

run_chatbot(user_message)